In [3]:
#Running on Chameleon GPUs - on a jupyter container connected to A100 compute gigaio

!nvidia-smi

Tue May  6 04:12:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:24:00.0 Off |                    0 |
| N/A   48C    P0             50W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
!pip install --upgrade \
  torch torchvision torchaudio \
  transformers \
  datasets[audio] \
  accelerate \
  soundfile \
  librosa \
  evaluate \
  jiwer \
  tensorboard \
  gradio

!pip install --upgrade --quiet pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 MB 9.4 MB/s eta 0:00:00:00:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 9.9 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 10.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 10.8 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import transformers
print(transformers.__version__)

4.51.3


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May  6 04:18:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:24:00.0 Off |                    0 |
| N/A   48C    P0             50W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [7]:
# 2) Imports and config
import random
import torch
from datasets import load_dataset, Audio, DatasetDict
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    pipeline,
)

In [13]:
import os
from datasets import Dataset
from tqdm import tqdm

data_dir = "/mnt/nptel-sample/paired"
samples = []

# Wrap with tqdm for progress
for fname in tqdm(os.listdir(data_dir), desc="Loading data"):
    if fname.endswith(".wav"):
        base = fname.replace(".wav", "")
        audio_path = os.path.join(data_dir, f"{base}.wav")
        text_path = os.path.join(data_dir, f"{base}.txt")

        if os.path.exists(text_path):
            with open(text_path, "r") as f:
                transcript = f.read().strip()
            samples.append({"audio": audio_path, "text": transcript})

ds = Dataset.from_list(samples)

Loading data: 100%|██████████| 2000/2000 [01:09<00:00, 28.63it/s]


In [15]:
ds[0]

{'audio': '/mnt/nptel-sample/paired/0000003b8fd9bc22877135b42b04c49d4860312b001be688723ecc5d.wav',
 'text': 'IN THIS PARTICULAR SESSION WE WILL BE DISCUSSING HOW TO BENCHMARK SOMETHING THAT I GOT TO'}

In [21]:
ds = ds.train_test_split(test_size=0.1, seed=42)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 900
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 100
    })
})

In [22]:
# ensure audio is resampled to 16 kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

In [23]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [24]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [25]:
print(ds['train'][0])
print(ds['test'][0])

{'audio': {'path': '/mnt/nptel-sample/paired/0008d1d2ea3a24d28dfa1a2d29c4aeb8bd7908fb14016d2e752c652e.wav', 'array': array([0.18383789, 0.1776123 , 0.17593384, ..., 0.01663208, 0.02108765,
       0.02206421]), 'sampling_rate': 16000}, 'text': 'THEN THIS PROCESS IS KNOWN AS FADING AND THEREFORE WE NEED MECHANISMS WHICH OVERCOME THIS SORT OF FADING EFFECT OF THE WIRELESS CHANNEL AND SORT OF ENSURE SERVICE OR GOOD QUALITY OF SERVICE'}
{'audio': {'path': '/mnt/nptel-sample/paired/000a9777bbfddbb1da493aae6c6ca41a7665e610ae872c7f887bfd88.wav', 'array': array([ 0.04421997,  0.04571533,  0.04522705, ..., -0.03311157,
       -0.02905273, -0.02542114]), 'sampling_rate': 16000}, 'text': 'HEADS APPEAR OK SO I TOSS A COIN AND UNLESS I GET TWO HEADS CONSECUTIVELY I'}


In [26]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [27]:
ds = ds.map(prepare_dataset, remove_columns=ds.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/900 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [29]:
model.generation_config.language = "English"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [30]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [32]:
import evaluate

metric = evaluate.load("wer")

In [33]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    print(f"WER: {wer:.2f}%")
    return {"wer": wer}

In [34]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [48]:
ds

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 900
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 100
    })
})

In [58]:
from transformers import Seq2SeqTrainingArguments

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-small-hi",
#     num_train_epochs=1,
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=1,
#     learning_rate=1e-5,
#     warmup_steps=500,
#     max_steps=4000,
#     # gradient_checkpointing=True,
#     fp16=True,
#     fp16_full_eval=True, #can use on A100 GPUs for faster evaluation
#     eval_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=False,
#     save_steps=2000,
#     eval_steps=2000,
#     logging_steps=25,
#     logging_strategy="steps",
#     report_to=None,
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
# )

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-fast",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=1e-5,
    logging_steps=25,
    logging_strategy="steps",
    save_steps=500,
    eval_strategy="steps",
    report_to=None,
    fp16=True,
    gradient_checkpointing=False,
    predict_with_generate=True,
    fp16_full_eval=True, #can use on A100 GPUs for faster evaluation
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [59]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer
)

# this is just an explicit callback in case report to "none" is still not printing anything to my console.

# from transformers import TrainerCallback

# class LossPrinterCallback(TrainerCallback):
#     def on_log(self, args, state, control, logs=None, **kwargs):
#         if logs is not None:
#             print(f"Step {state.global_step} - {logs}")

# trainer.add_callback(LossPrinterCallback())

/tmp/ipykernel_264/3278066946.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [56]:
processor.save_pretrained(training_args.output_dir)

[]

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
25,0.132600,0.416752,56.536697
50,0.278400,0.402919,31.364679
75,0.160700,0.404685,55.275229
100,0.108900,0.404472,55.275229


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


WER: 56.54%
WER: 31.36%
WER: 55.28%
WER: 55.28%


In [ ]:
# trainer.save_model("whisper-small-hi-finetuned")
# processor.save_pretrained("whisper-small-hi-finetuned")

In [ ]:
import torchaudio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def transcribe(audio_path):
    speech, sr = torchaudio.load(audio_path)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        speech = resampler(speech)

    inputs = processor(speech.squeeze(), sampling_rate=16000, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # move to GPU if available

    with torch.no_grad():
        predicted_ids = model.generate(inputs["input_features"])
        transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription

# Run
print(transcribe("000ace94c86177e4733f35dee57be61fda36484ad5481ab95f3694e2.wav"))